In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import threading 
from datetime import datetime

## Group Assignment
### Team Number: 01
### Team Member Names: Saige Belanger, Hugh Jiang, Steven Mou
### Team Strategy Chosen: SAFE

## Explanation of Strategy

### Mean Returns, Standard Deviation, and Beta

The mean return of a stock is the average return that a stock yields. Also known as expected return, we could use the mean return to estimate the profit or loss the stock would give us. As according to our "SAFE" strategy, we want our returns to remain as close to zero as possible. As stock prices are constantly changing, it is almost impossible to find one stock that would have a return of zero — let alone the minimum of ten stocks we need for our portfolio. Thus, to keep returns as close to zero as possible, we correspondingly wanted stocks which had the closest mean returns to zero when compared to the rest of the stocks. As we know that the mean returns can be close to zero, both as profits and losses, we sorted a list of tickers (ascending order) based on the absolute value of the difference between 0 and the mean return of the ticker (Distance from 0). The tickers at the front of the sorted list would be the tickers we require from the stocks' mean returns. 

The standard deviation for a stock is a statistical measurement in finance that tells us how volatile a stock is, historically. Whereas a volatile stock would have a higher standard deviation, a stable stock would have a lower standard deviation. Since our team's strategy is "SAFE", what we needed were more stable stocks — stocks with lower standard deviations. We need stocks with lower volatility because it would mean the stock prices fluctuate less and tend to be more stable relative to its expected return. We sorted a list of tickers (ascending order) based on the standard deviation values of each ticker. Thus, the tickers at the front of the sorted list tell us the tickers with the lowest standard deviation when compared to the rest of the tickers. 

The beta of a stock tells us the volatility of that stock when compared to the overall market. By definition, the beta of the overall market is 1.0. If the beta of the stock is greater than 1.0, then the stock fluctuates more than the market. We want our beta to be lower than 1.0, which would mean the stock is more stable compared to the market. We were able to get the betas of each stock by directly extracting the betas from the yfinance database. Although we cannot guarantee that at least ten stocks' betas would be less than 1.0, we still sorted the list of tickers based on the beta by ascending order. Like our sorted lists for mean returns and standard deviation, the tickers in the front of the sorted beta list are the tickers we need. 

Since mean return, standard deviation and beta are all important things to consider when creating a "SAFE" portfolio, we wanted ten stocks that would have the lowest values for all three relative to the rest of the stocks given. To find those ten stocks, we used our function `positioned-list` to combine all three of the ordered lists. The method of ordering for `positioned-list` relies on the sum of the index positions for each ticker in two ordered lists. The ticker with the lowest sum would be placed first in the list, while the ticker with the highest sum would be placed last. 

We first combined the the ordered list for mean returns and the ordered list for standard deviation in the `stdd-returns` list. We needed these two metrics to be equally weighted in the final combined list, so they were combined together first. Standard deviation and expected returns are closely related, and we wanted stocks with low expected returns *and* small fluctuations relative to those small expected returns with an equal weight for both.

To follow, we combined `stdd-returns` with the ordered list for betas to create `stdd-returns-betas` list. By combining the ordered beta list second, this allows us to implement 50% weighting on beta, while both standard deviation and mean returns receive 25% weighting each. `stdd-returns-betas` list is our finalized ordered list that chooses ten stocks when considering close-to-zero mean returns, low standard deviations, and low betas.

This was the first half of our strategy for choosing stocks.

### Mean Returns, Standard Deviation and Correlation

The correlation between two stocks tell us how the two stocks move in relation to each other. A correlation of 1.0 means that the two stocks move in the same direction and in perfect tandem to one another. As such, a correlation of -1.0 means that the stocks move perfectly opposite to one another. To diversify our portfolio (reduce non-market risk) and keep our returns close to zero, we would need stocks that have negative correlation to each other. If one stock does well, it could at least somewhat be offset by the other stock which could be doing poorly, thus keeping returns closer to zero. However, as we know, stocks with negative correlation to one another are  uncommon. Hence, we simply need the lowest correlation values possible. With a lower correlation value, even if one stock was performing significantly well or significantly poorly, the other stock would not have been performing as well or as poorly, respectively. Hence, making lower profits or losses rather than greater profits or losses. To find the stocks with lowest correlations to one another, we created a correlation matrix. 

This strategy has the same idea as the first strategy. However, the approach is slightly different since correlation requires the comparison between the data of two stocks. For each ticker in the `stdd-returns-betas` list, we sorted a list of tickers with the lowest correlation to each given ticker. The first ticker in each sorted correlation list would be added into the finalized list of ten tickers for our second strategy, which has a 50% weighting on correlation, 25% on standard deviation, and 25% on returns. Since there is the possiblity of a single ticker having lowest correlation to multiple tickers, we implemented in our code to remove any duplicate tickers. 

### Final List of Tickers Chosen

To get our final list of tickers, we combined our list of tickers `tickers_group_1` from our first strategy (beta, mean returns, standard deviation) and our list of tickers `tickers_group_2` from our second strategy (correlation, mean returns, standard deviation). The reason for our two strategies is because neither strategy guarantees an infallible success. Just like a portfolio, our portfolio-picking strategies need to be diversified as well. 

The reason for choosing ten stocks for each strategy is done in accordance to the '# of stocks chosen' requirement. Similar to the second strategy, our finalized list of tickers could also contain duplicate tickers. Even if every ticker in the first strategy had a duplicate from the list in the second strategy, there would still be the minimum of ten stocks chosen after our code removes duplicates. Furthermore, even if every ticker in the first strategy did not have a duplicate from the list in the second strategy, it would still meet the requirement of maximum of 20 stocks. 

## Weightings of Tickers

To choose appropriate weightings such that our portfolio generates returns closest to zero, we simulated several portfolios with our final list of chosen tickers and placed each portfolio into a DataFrame. Of course, to meet requirements, we skipped portfolios where any single ticker did not make up the minimum/maximum weights of (100/(2n))% and 35% respectively. We then chose the portfolio with the average daily return closest to zero. 

## That Would Be Our Final Portfolio !!!

## Functions

In [2]:
def csv_to_list(file_name):
    '''
    Produces a list of tickers in the given csv file
    
    Params:
        file_name (Str): file name of the tickers csv file being imported
    '''
    tickers = [] # list of tickers

    csv = pd.read_csv(file_name) # read the csv file

    # The first ticker entry in the csv is read in as the column title, so add the ticker manually to the list of tickers and then sort the list
    first_stock = csv.columns[0]
    tickers = list(csv.iloc[:,0])
    tickers.append(first_stock)
    tickers.sort()
    
    return tickers

In [3]:
def filter_tickers(tickers):
    '''
    Consumes a list of string tickers and produces a new list of tickers that contain valid data and meet exchange/minimum volume requirements.
    
    Params:
        tickers (list[Str]): List of String Tickers 
        
    Returns: 
        list[Str]: New Filtered Tickers in alphabetical order
    '''
    
    min_avg_daily_volume = 10000       # Minimum average daily volume must be at least this number of shares
    valid_exchanges = ['NYQ', 'NMS']   # We only want american exchanges

    new_tickers = []         # filtered tickers list
    ticker_info_dicts = []   # List of Ticker Info Dictionaries
    
    # Threading variables
    threads = []
    lock = threading.Lock()
    
    def add_ticker_dict(ticker):
        stock = yf.Ticker(ticker) # Ticker object
        
        # Get average daily volume (shares/day)
        avg_daily_vol = stock.history(start='2021-07-02', end='2021-10-22')['Volume'].mean()
        
        # Check if the stock meets minimun volumne requirements from July 02, 2021 to October 22, 2021.
        if avg_daily_vol >= min_avg_daily_volume:
            
            info_dict = yf.Ticker(ticker).info # Get the stock info dictionary
            
            # If we meed minimum volume requirements, check if we meet exchange location requirements
            if info_dict.get('exchange') in valid_exchanges:
                with lock:
                    # Add ticker to a tickers list
                    new_tickers.append(ticker)
                    # print(ticker + " is valid")

    for ticker in tickers:
        t = threading.Thread(target=add_ticker_dict, args=(ticker,))
        threads.append(t)

    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()
    
    # Sort tickers list and return
    new_tickers.sort()
    
    print('Successfully added all valid tickers to a string list. If invalid or data not found, the ticker is removed.')
    return new_tickers

In [4]:
def get_ticker_objects(tickers):
    ''' 
    Consumes a list of ticker strings and produces a list containing YFinance Ticker Objects.
    '''
    ticker_objects = []
    for ticker in tickers:
        ticker_objects.append(yf.Ticker(ticker))
    
    return ticker_objects

In [5]:
def get_info_dicts(tickers, ticker_objects):
    '''
    Consumes a list of ticker strings a list of yfinance ticker objects, and returns a dictionary containing stock info 
    dictionaries, where the keys are stock ticker symbols.
    
    Params:
        tickers (list[Str])
        
    Returns:
        dict[Str, dict]
    
    Example Return Data:
    {
        'AAPL': {... info dict ...},
        'AMC': {... info dict ...},
        'SHOP': {... info dict ...}
    }
    '''
    lock = threading.Lock()    
    
    info_dicts = {}
    
    def get_ticker_info(index):
        stock = ticker_objects[index] # yf finance ticker object
        info_dict = stock.info
        
        with lock:
            info_dicts[tickers[index]] = info_dict
    
    threads = []
    lock = threading.Lock()
    for i in range(len(tickers)):
        t = threading.Thread(target=get_ticker_info, args=(i,))
        threads.append(t)
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()  
    
    print('Successfully added info dictionaries to a new dictionary containing string ticker keys')
    return info_dicts

In [6]:
def get_betas(tickers, info_dicts):
    '''
    Consumes a string list of tickers and a dictionary of stock info dictionaries, and returns a list of tickers sorted in order of increasing beta.
    
    Example Return: ['AAPL', 'SHOP', ...]
    '''
    beta_frame= pd.DataFrame(new_tickers)
    beta_frame['Beta']= ""
    for i in range (len(new_tickers)):
        beta_frame.iloc[i,1]= info_dicts[beta_frame.iloc[i,0]]['beta']
    
    sorted_betas= beta_frame.sort_values(by= 'Beta', ascending=True)
    
    beta_sorted_list= sorted_betas.iloc[:,0].to_list()

    return beta_sorted_list

In [7]:
def positioned_list(ticker_list, ordered_list1, ordered_list2):
    '''
    Consumes a list of tickers and two ordered lists of those same tickers, and combines the lists based on position in the two ordered lists. 
    
    Params:
        ticker_lst: list of tickers in both lists in any order
        ordered_list1: first ordered list of tickers
        ordered_list2: second ordered list of tickers
    '''
    pos_df= pd.DataFrame(ticker_list)

    pos_df['Beta Position']= ""
    pos_df['STD Position']= ""

    for object in (ticker_list):
        pos_df.iloc[ticker_list.index(object),1]= ordered_list1.index(object)
        pos_df.iloc[ticker_list.index(object),2]= ordered_list2.index(object)
        pos_df['Combined Positioning']= ""

    for i in range (len(ticker_list)):
        pos_df.iloc[i,3]= pos_df.iloc[i,1] + pos_df.iloc[i,2]

    sorted_position= pos_df.sort_values(by= 'Combined Positioning', ascending=True).iloc[:,0].to_list()

    return sorted_position

## Code

#### Import YFinance Data

Import tickers csv, filter out tickers based on assignment requirements, and then save a list of ticker objects and info dictionaries to variables (so we don't need to continue calling the Yahoo Finance API).

**Important Variables**:

`new_tickers` - list of tickers that fulfill minimum volume and exchange location requirements

`ticker_objects` - list of YFinance Ticker objects corresponding to the tickers in `new_tickers`

`info_dicts` - stores info dictionaries for each ticker from YFinance

In [8]:
# Import tickers list
tickers = csv_to_list("Tickers.csv") 



# NEW CODE

# Remove duplicates from tickers list
tickers = list(set(tickers))

# END OF NEW CODE



# Filter tickers list and get new list with valid tickers based on assignment requirements
new_tickers = filter_tickers(tickers) 

# List of ticker corresponding to the tickers in "new_tickers" list
ticker_objects = get_ticker_objects(new_tickers)

# Info dictionaries saved as a dictionary where the key is the ticker symbol, the info dictionary is the value corresponding to the key
info_dicts = get_info_dicts(new_tickers, ticker_objects)

- CELG: No data found, symbol may be delisted
- INVALIDTIC: No data found, symbol may be delisted
- AUST: No data found for this date range, symbol may be delisted
Successfully added all valid tickers to a string list. If invalid or data not found, the ticker is removed.
Successfully added info dictionaries to a new dictionary containing string ticker keys


#### Add Price History and Returns to a DataFrame
Create a dataframe <code>prices</code> that stores daily close price history for all our tickers, and another dataframe <code>daily_returns</code> that stores daily percentage returns of the tickers (in decimal percent).

We decided to take 5 years of historical daily data for our analysis because we found it was a good balance of speed and data volume (in general, the more data we have, the more accurate/consistent our statistical analysis).

In [9]:
# Set start and end dates for historical data
data_start_date = '2016-11-26'
data_end_date = '2021-11-27'

In [10]:
#DataFrame to store Closing Prices
prices = pd.DataFrame()

#Adds Closing prices of tickers to a DataFrame
for i in range(len(ticker_objects)):
    stock = new_tickers[i]
    stock_hist = ticker_objects[i].history(start=data_start_date, end=data_end_date)
    stock_hist = stock_hist['Close']
    prices[stock] = stock_hist
    
prices.head()

,AAPL,ABBV,AMZN,AXP,BAC,BK,BMBL,CMCSA,COST,CSCO,...,ORCL,OXY,PEP,PYPL,SHOP,SLB,SO,SPG,SQ,VZ
Date,,,,,,,,,,,,,,,,,,,,,
2016-11-28,26.332415,46.565838,766.770020,66.966866,18.265123,41.683708,NaN,31.116854,135.241028,25.781096,...,37.145752,55.977673,89.289146,39.959999,42.490002,68.083633,38.487396,142.083832,12.53,40.926723
2016-11-29,26.306452,48.233952,762.520020,66.372681,18.256128,41.701439,NaN,31.950455,135.446732,25.703545,...,37.376179,55.306503,89.071983,39.889999,42.590000,67.126114,38.487396,142.843918,12.46,40.798626
2016-11-30,26.084597,47.615257,750.570007,66.883331,19.073429,42.029373,NaN,31.663490,134.221741,25.694927,...,37.044357,58.408630,87.595726,39.279999,41.669998,70.594971,37.300339,140.767441,12.94,39.949993
2016-12-01,25.841503,46.330898,743.650024,67.338234,19.416605,43.075226,NaN,31.822914,135.679184,25.376112,...,35.670986,58.334953,86.659409,38.340000,39.669998,71.065331,37.117111,138.549927,12.71,39.925980
2016-12-02,25.938267,46.542355,740.340027,66.716187,19.172770,42.233223,NaN,31.330942,135.992142,25.203777,...,35.486633,57.991196,88.033264,38.619999,39.049999,71.401306,37.252541,140.540207,13.06,39.877945


In [11]:
#Calculates Daily Returns from DataFrame
daily_returns = prices.pct_change()
daily_returns.head()

,AAPL,ABBV,AMZN,AXP,BAC,BK,BMBL,CMCSA,COST,CSCO,...,ORCL,OXY,PEP,PYPL,SHOP,SLB,SO,SPG,SQ,VZ
Date,,,,,,,,,,,,,,,,,,,,,
2016-11-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-11-29,-0.000986,0.035823,-0.005543,-0.008873,-0.000492,0.000425,NaN,0.026789,0.001521,-0.003008,...,0.006203,-0.011990,-0.002432,-0.001752,0.002353,-0.014064,0.000000,0.005350,-0.005587,-0.003130
2016-11-30,-0.008433,-0.012827,-0.015672,0.007694,0.044769,0.007864,NaN,-0.008982,-0.009044,-0.000335,...,-0.008878,0.056090,-0.016574,-0.015292,-0.021601,0.051677,-0.030843,-0.014537,0.038523,-0.020801
2016-12-01,-0.009319,-0.026974,-0.009220,0.006801,0.017992,0.024884,NaN,0.005035,0.010858,-0.012408,...,-0.037074,-0.001261,-0.010689,-0.023931,-0.047996,0.006663,-0.004912,-0.015753,-0.017774,-0.000601
2016-12-02,0.003745,0.004564,-0.004451,-0.009238,-0.012558,-0.019547,NaN,-0.015460,0.002307,-0.006791,...,-0.005168,-0.005893,0.015854,0.007303,-0.015629,0.004728,0.003649,0.014365,0.027537,-0.001203


#### Get Returns

We want stocks with returns that are close to zero. To do this, we can determine the mean daily return of each stock and take the magnitude (absolute value) of the mean return.

Then, we sort a list of tickers <code>sorted_returns</code> in order of increasing magnitude of expected returns. The tickers at the beginning of the list are most desirable based on returns, whereas the tickers near the end of the list are the least desirable.

In [12]:
# Get the mean return of each stock in a dataframe
sorted_returns_df = pd.DataFrame(daily_returns.mean(), columns=['mean_return'])

# Compute the distance of returns from zero (note that we want returns closest to zero)
sorted_returns_df['distance_from_zero'] = abs(0-sorted_returns_df['mean_return'])

# Sort in ascending order by distance of returns from zero
sorted_returns_df = sorted_returns_df.sort_values(by='distance_from_zero')
sorted_returns_df.head()

,mean_return,distance_from_zero
IBM,0.000085,0.000085
HOOD,0.000140,0.000140
DUOL,0.000146,0.000146
OXY,0.000222,0.000222
VZ,0.000256,0.000256


In [13]:
# Get a list of stocks with the lowest mean returns in magnitude (returns closest to zero)
sorted_returns = sorted_returns_df.index.to_list()

# display first 12 values 
sorted_returns[:12]

['IBM',
 'HOOD',
 'DUOL',
 'OXY',
 'VZ',
 'SLB',
 'BK',
 'CVS',
 'SO',
 'SPG',
 'CMCSA',
 'PEP']

#### Get Standard Deviations

We also want stocks with a low risk, i.e. low standard deviation. We take the standard deviation of each ticker's daily returns and then sort a list of tickers <code>sorted_stdd</code> that contains the tickers in order of increasing standard deviation. 

Again, tickers near the beginning of the list are more desirable with respect to standard deviation, whereas those near the end are less desirable.

In [14]:
#Gets Standard Deviation for each ticker
daily_returns_stdd = daily_returns.std()

#Stores Standard Deviation for each ticker in a DataFrame
df= pd.DataFrame(daily_returns_stdd,columns = ['Standard Deviation'])
#Sorts DataFrame based on value of Standard Deviation (Ascending order)
df= df.sort_values(by = 'Standard Deviation', ascending = True)

#List of tickers sorted based on value of Standard Deviation (Ascending order)
sorted_stdd = df.index.values.tolist()

# Display the first 12 tickers in the list
sorted_stdd[:12]

['VZ',
 'PEP',
 'COST',
 'SO',
 'IBM',
 'CMCSA',
 'ORCL',
 'GOOG',
 'CSCO',
 'ABBV',
 'CVS',
 'BK']

#### Get Betas

As already explained, we want stocks with lower betas. Similarly to what we did for returns and standard deviation, we create a list of tickers <code>sorted_betas</code> in order of increasing beta. 

Beta values are retreived by our <code>get_betas()</code> function which query Yahoo Finance's API for beta values for each ticker. According to Yahoo Finance, these beta values are based on the past 5 years of historical data, which is the same historical time frame we are using for the rest of our analysis. 

In [15]:
sorted_betas = get_betas(new_tickers, info_dicts)

# Display the first 12 tickers in the list
sorted_betas[:12]

['SO',
 'VZ',
 'PEP',
 'COST',
 'ABBV',
 'ORCL',
 'CVS',
 'CSCO',
 'GOOG',
 'CMCSA',
 'BK',
 'PYPL']

#### Make a Correlation Matrix

We use correlation to diversify our portfolio and reduce non-market risk in our portfolio. 

To do so, we first need to retreive correlation coefficients between each ticker. We create a correlation matrix <code>corr_matrix</code> that contains the correlation of each stock's *daily returns* with each other. Notice that we only display the first 5 rows of the matrix, but it contains correlation values for all tickers in our list. 

In [16]:
# Make a correlation matrix
corr_matrix = daily_returns.corr()

# Display first 5 rows of matrix
corr_matrix.head()

,AAPL,ABBV,AMZN,AXP,BAC,BK,BMBL,CMCSA,COST,CSCO,...,ORCL,OXY,PEP,PYPL,SHOP,SLB,SO,SPG,SQ,VZ
AAPL,1.000000,0.365916,0.634681,0.436074,0.441265,0.365472,0.322055,0.441332,0.499982,0.575969,...,0.491756,0.274456,0.484622,0.607288,0.465812,0.316021,0.325236,0.272853,0.514673,0.254956
ABBV,0.365916,1.000000,0.300611,0.358602,0.367185,0.376443,0.041465,0.347514,0.305601,0.448258,...,0.379110,0.262956,0.389575,0.338670,0.263563,0.318756,0.267434,0.252842,0.274736,0.336941
AMZN,0.634681,0.300611,1.000000,0.260883,0.271515,0.230073,0.318139,0.331645,0.435020,0.464209,...,0.386600,0.156982,0.319603,0.572473,0.539177,0.194312,0.155180,0.098145,0.530662,0.158527
AXP,0.436074,0.358602,0.260883,1.000000,0.785438,0.710073,0.107848,0.522819,0.289480,0.533428,...,0.479593,0.548345,0.443517,0.398284,0.214518,0.644710,0.441671,0.647221,0.363929,0.356967
BAC,0.441265,0.367185,0.271515,0.785438,1.000000,0.802180,0.083320,0.517728,0.306383,0.559990,...,0.497541,0.586212,0.419133,0.356817,0.172751,0.643108,0.382140,0.546232,0.317573,0.382584


#### Combine Stock Ticker Lists

For a safe portfolio, we want a portfolio of stocks with low risk and low returns. That is, we want a combination of low standard deviation (risk), low returns, and low beta (relative risk to the market). We also want a diversified portfolio to reduce non-market risk, which we can do by picking stocks that have low correlation coefficients with each other. 

To combine these metrics together, we use our `positioned_list` function (explained earlier) to get tickers with the lowest returns, standard deviations, and betas relative to the other tickers. We first combine the lowest returns and standard deviations list to get them each weighted 25%, and combine the betas list with that to get betas weighted 50%. 

The top 10 stocks in this combined list will make up the first group of 10 stocks that we pick, stored in a list `tickers_group_1`. The next group of at most 10 stocks will be chosen based on the stocks with the lowest correlation to our first group (50% weighting), the stocks with the lowest returns (25% weighting) and those with the lowest standard deviation (25% weighting). This tickers list is stored in `tickers_group_2`.

**Note:** *The more detailed reasoning behind this technique was explained at the top of our file.*

In [17]:
stdd_returns = positioned_list(new_tickers, sorted_stdd, sorted_returns)
stdd_returns_betas = positioned_list(new_tickers, stdd_returns, sorted_betas)
stdd_returns_betas[:10]

['VZ', 'SO', 'PEP', 'ORCL', 'COST', 'CVS', 'CMCSA', 'ABBV', 'BK', 'CSCO']

In [18]:
tickers_group_1 = stdd_returns_betas[:10]
tickers_group_1

['VZ', 'SO', 'PEP', 'ORCL', 'COST', 'CVS', 'CMCSA', 'ABBV', 'BK', 'CSCO']

In [19]:
# Get the second group of tickers based on tickers with the lowest corr and stdev

tickers_group_2 = []

for ticker in tickers_group_1:
    # Get list of stock tickers with lowest correlation with the given ticker
    sorted_corrs = corr_matrix[ticker].sort_values().index.to_list()
    
    # Combine the list of lowest correlation tickers with a list of lowest standard deviations and returns 
    # 50% weighting for correlation, 25% for stdev, 25% for returns
    combined_list = positioned_list(new_tickers, sorted_stdd, sorted_returns)
    combined_list = positioned_list(new_tickers, combined_list, sorted_corrs)
    
    # Get the first stock in this combined list and add it to tickers_group_2
    tickers_group_2.append(combined_list[0])
    
# Remove duplicates from tickers_group_2
tickers_group_2 = list(set(tickers_group_2))
tickers_group_2

['SO', 'BK', 'HOOD', 'VZ']

Now that we have both lists of tickers, we can combine them into our final tickers list `final_tickers` (excluding any duplicate tickers).

In [20]:
# Get final tickers list by combining tickers_group_1 and tickers_group_2 and removing duplicate tickers
final_tickers = list(set(tickers_group_1 + tickers_group_2))
final_tickers.sort()
final_tickers

['ABBV',
 'BK',
 'CMCSA',
 'COST',
 'CSCO',
 'CVS',
 'HOOD',
 'ORCL',
 'PEP',
 'SO',
 'VZ']

### Weights

Now, we simulate ~1m different ticker weightings and choose the portfolio with the lowest return. 

In [21]:
final_tickers_mean= daily_returns[final_tickers].mean()
final_tickers_mean_df= pd.DataFrame(final_tickers_mean)
final_tickers_mean_df.rename(columns= {0:'mean_returns'}, inplace=True)

In [22]:
num_simulations = 1000000

portfolio_returns=[]
portfolio_std=[]
portfolio_weights=[]

for i in range (num_simulations):
    weights= np.random.random(len(final_tickers))
    weights= weights/sum(weights)
    
    # If any of the random weights are less than the min or greater than the max weight, skip the loop iteration
    if weights.min() < (1/(2*len(final_tickers))) or weights.max() > 0.35:
        continue

    portfolio_weights.append(weights)
    returns=(weights @ final_tickers_mean_df['mean_returns'])
    portfolio_returns.append(returns)

# Get the distance of the portfolio returns from zero (since we want the returns closest to zero)
portfolio_returns = [abs(p_return) for p_return in portfolio_returns]

data= {'portfolio_return': portfolio_returns} # Average daily return of overall portfolio

for i, symbol in enumerate(final_tickers):
    data[symbol]= [w[i] for w in portfolio_weights]

different_weights = pd.DataFrame(data)
different_weights = different_weights.sort_values(by='portfolio_return')
different_weights.head()

# 
# portfolio_return is the MAGNITUDE of average daily returns of the simulated portfolio. Other columns are the simulated stock weights.
# The dataframe has been sorted so the optimal portfolio (with the return closest to zero) is the first row. 

,portfolio_return,ABBV,BK,CMCSA,COST,CSCO,CVS,HOOD,ORCL,PEP,SO,VZ
11432,0.000487,0.052305,0.071329,0.053476,0.059133,0.050730,0.053522,0.189883,0.059806,0.169640,0.068856,0.171321
13967,0.000489,0.047647,0.046263,0.060340,0.056043,0.072932,0.047365,0.192943,0.082813,0.080404,0.123773,0.189476
12547,0.000491,0.072792,0.138966,0.071896,0.049209,0.052245,0.047736,0.157136,0.050410,0.068003,0.140337,0.151269
5745,0.000495,0.049075,0.188258,0.085058,0.046618,0.068177,0.046935,0.189091,0.090848,0.050804,0.082559,0.102578
7599,0.000495,0.072898,0.150474,0.047221,0.050660,0.054271,0.096602,0.184472,0.068050,0.077252,0.088566,0.109534


Each row in the above DataFrame `different_weights` is a different simulated portfolio with different stock weights.

The first column of `different_weights` is specifically to show the magnitude of the mean daily returns of the portfolio, and the rest of the columns are to show the weightings for each of the tickers. The completed DataFrame with all the portfolios is sorted (according to the magnitude of mean daily returns) so that the first row of the DataFrame reveals the portfolio with the returns closest to zero. Using this first row, we will have our final weightings for each of the final tickers chosen for our final portfolio. 

In [23]:
# Display optimal weights
different_weights[final_tickers].iloc[0]

ABBV     0.052305
BK       0.071329
CMCSA    0.053476
COST     0.059133
CSCO     0.050730
CVS      0.053522
HOOD     0.189883
ORCL     0.059806
PEP      0.169640
SO       0.068856
VZ       0.171321
Name: 11432, dtype: float64

#### Add Data to final dataframe "FinalPortfolio"

Now that we have our final tickers and weightings for those tickers, we have all the information we need to generate a dataframe to display our final portfolio. 

The first column of the DataFrame displays all the tickers we want to use for the final portfolio. The second column displays each ticker's closing stock price. The third column displays the # of shares for each ticker in which we would own in our final portfolio. To calculate this, we multiplied each ticker's weighting by the \\$100000 investment and divided the result of that by the ticker's respective share prices. The fourth column displays how much value($) of each ticker we own within our portfolio, and we calculated this by finding the product of the ticker's share price and the amount of shares we own for that ticker. The fifth and final column shows the tickers' weightings. 

In [24]:
# FinalPortfolio Dataframe
FinalPortfolio = pd.DataFrame()
FinalPortfolio['Ticker'] = final_tickers
FinalPortfolio['Price'] = list(prices[final_tickers].iloc[-1])
FinalPortfolio['Shares'] = list(100000*different_weights[final_tickers].iloc[0])
FinalPortfolio['Shares'] = FinalPortfolio['Shares']/FinalPortfolio['Price'] 
FinalPortfolio['Value'] = FinalPortfolio['Shares']*FinalPortfolio['Price']
FinalPortfolio['Weight'] = list(different_weights[final_tickers].iloc[0])
FinalPortfolio.index += 1 # Start the index at 1

print(f"Sum of Values: ${FinalPortfolio.Value.sum()}")
print(f"Sum of Weights: {FinalPortfolio.Weight.sum()*100}%")

Sum of Values: $99999.99999999999
Sum of Weights: 99.99999999999997%


In [25]:
FinalPortfolio

,Ticker,Price,Shares,Value,Weight
1,ABBV,116.510002,44.892847,5230.465688,0.052305
2,BK,56.750000,125.689255,7132.865238,0.071329
3,CMCSA,51.099998,104.649546,5347.591628,0.053476
4,COST,546.130005,10.827694,5913.328745,0.059133
5,CSCO,54.669998,92.792857,5072.985329,0.050730
6,CVS,91.519997,58.480960,5352.177259,0.053522
7,HOOD,27.920000,680.095370,18988.262784,0.189883
8,ORCL,92.330002,64.774518,5980.631366,0.059806
9,PEP,161.139999,105.274911,16963.999137,0.169640
10,SO,62.040001,110.986806,6885.621523,0.068856


In [26]:
# Create Stocks dataframe and export csv
Stocks = FinalPortfolio[['Ticker', 'Shares']]
Stocks.to_csv('Stocks_Group_01.csv')

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Saige Belanger, Hugh Jiang, Steven Mou

## New Code Explanation

<span style="color: red;">Our original code did not run properly because we didn't anticipate the inclusion of duplicate tickers in the tickers file, which lead to an error in the positioned_list function. (Dataframe row indexes were set with the ticker string name, but duplicate tickers lead to duplicate row indexes. When we populated the dataframe with data, the second duplicated row had empty data and resulted in errors.)</span> 

<span style="color: red;">To fix this, we added a line that removes duplicates from the ticker list when we import from Tickers.csv.</span> 